In [ ]:
import os
import time
import math
import random
import numpy as np
import pandas as pd
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [ ]:
random.seed(42)
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
prompt_template = ''

In [4]:
def build_prompt(text):
    return prompt_template.replace('KALIMAT:', text)

In [5]:
def load_data():
    data = pd.read_csv('./val.csv', encoding='latin-1')
    return data

In [6]:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
]

def generate_noise(prompt):
    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-04-17", 
        # model='gemini-2.0-flash',
        contents=prompt, 
        config=types.GenerateContentConfig(
            safety_settings=safety_settings,
        )
    )
    return response.text

# CHAR LEVEL NOISE

In [ ]:
# Typos
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mensimulasikan typo pada kalimat tersebut.
- lakukan pada maksimal 5-10 kata
- output hanya berupa kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil'''

In [ ]:
data = load_data()

In [ ]:
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['processed_text']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["processed_text"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

Index: 1
Original: url menang berkat dua pemain muslim dan url enam pemain muslim yang membawa madrid juara dan url pria muslim di balik kemenangan memang madrid di final liga champions dan njink untung saudick kalah coba kalau menang pasti bawa bawa agama tertawa tolol dan nbelum lupa tahun dulu dan url pemain kafir heran arab saudi muslim dan tertawa 

Augmented: url menang berkatt dua pemain muslm dan url enaam pemain musliim yang membawa marid juara dan url pria muslim di balik kemengana memang madrid di final liga champons dan njink untung saudik kalah coba kalau menang pasti bawa bawaa agama tertawa tool dan nbelum lupa tahun dulu dan url pemain kafir heran arab saudi muslim dan tertawa
Index: 7
Original: merawat energi al maidah untuk kebangkitan umat demi kejayan negara kesatuan republik indonesia h maulid agung dan reuni akbar alumni reuniakbaralumnipic twiter com wvkfzejvwn 

Augmented: merawwat ennergi al maidah untuk kebanngkitan umatt demi kejayyan neggara kesatuan republl

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash', 'location': 'global'}, 'quotaValue': '10'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '59s'}]}}

In [ ]:
augmented_cnt = 110

for i in range(1139, len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['processed_text']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["processed_text"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(5)

Index: 1147
Original: heh kontol jan sok pemes kamu di rp muka seperti memek orang tua kamu seperti monyet agama kamu aje agama kontol 

Augmented: heh kontol gan sokj pems kemu di rp mjka seperti memekj orang tua kamu seperti monyet agmaa kamu aje agama kotol
Index: 1148
Original: ah oh ah ah honeyrt oh kamu keluarin sayang kontol aku masik lagi di memek kamu ah oh ah 

Augmented: ah oh ah ah honeeyrt oh kmau keluariin sayangg konttol akuh masiiik lagii d mmemek kmau ah oh ah
Index: 1164
Original: ibu saja kalau beli telur pakai milih sebelum ditimbang pdh telur kan mirip ibu itu tahu telur ada yang busuk 

Augmented: ibu sja kalu bli telor pkai mili sblum ditinbang pdh telur kan mirip ibuk itu tahu telur ada yang busuk
Index: 1167
Original: kalau kata melani mah kalau yang asli sama palsu tu beda kena tipu lagi barang palsu 

Augmented: kalau kta melan mah kalau yang asil sam palssu tj bda kna tipu lagi barang palsu
Index: 1173
Original: melayani tante kesepian area sby pesan langsun

In [163]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 151


In [ ]:
data.to_csv('./char-level-noise.csv', index=False)

# SYNONYM REPLACEMENT

In [230]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mengganti maksimum 10 kata dengan sinonim nya
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [232]:
data = load_data()
data.head()

,original_text,source,pornografi,sara,radikalisme,pencemaran_nama_baik,processed_text
0,Kalo urusan THR kompak yee. Gak ada kafir kafi...,twitter,0,1,0,0,kalau urusan thr kompak ya tidak ada kafir kaf...
1,[url=&quot;https://m.republika.co.id/amp_versi...,kaskus,0,1,0,0,url menang berkat dua pemain muslim dan url en...
2,Pantatnya ga sempurna banyak lemak Haha.. Kata...,instagram,1,0,0,1,pantatnya tidak sempurna banyak lemak tertawa ...
3,\n\nSalut buat para saudara saudara ku yang be...,kaskus,0,0,1,0,dan nsalut buat para saudara saudara ku yang b...
4,"Yah lu min,, lu mau goblokin org,, tpi lu mala...",instagram,0,0,0,1,yah kamu administrator kamu mau goblokin orang...


In [233]:
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['processed_text']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["processed_text"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(5)

Index: 1
Original: url menang berkat dua pemain muslim dan url enam pemain muslim yang membawa madrid juara dan url pria muslim di balik kemenangan memang madrid di final liga champions dan njink untung saudick kalah coba kalau menang pasti bawa bawa agama tertawa tolol dan nbelum lupa tahun dulu dan url pemain kafir heran arab saudi muslim dan tertawa 

Augmented: url jaya sebab dua pemain muslim dan url enam pemain muslim yang mengantar madrid kampium dan url lelaki muslim di belakang kejayaan sungguh madrid di final liga champions dan njink untung saudick takluk coba kalau menang pasti bawa bawa agama tergelak tolol dan nbelum lupa tahun dulu dan url pemain kafir heran arab saudi muslim dan tertawa
Index: 7
Original: merawat energi al maidah untuk kebangkitan umat demi kejayan negara kesatuan republik indonesia h maulid agung dan reuni akbar alumni reuniakbaralumnipic twiter com wvkfzejvwn 

Augmented: menjaga kekuatan al maidah bagi kebangunan rakyat demi kemasyhuran negeri persatu

In [234]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 151


In [235]:
data.to_csv('./noisy/synonym-replacement.csv', index=False)

In [ ]:
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['processed_text']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["processed_text"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(5)

# WORD DELETION

In [6]:
data = load_data()
data.head()

,original_text,source,pornografi,sara,radikalisme,pencemaran_nama_baik,processed_text
0,Kalo urusan THR kompak yee. Gak ada kafir kafi...,twitter,0,1,0,0,kalau urusan thr kompak ya tidak ada kafir kaf...
1,[url=&quot;https://m.republika.co.id/amp_versi...,kaskus,0,1,0,0,url menang berkat dua pemain muslim dan url en...
2,Pantatnya ga sempurna banyak lemak Haha.. Kata...,instagram,1,0,0,1,pantatnya tidak sempurna banyak lemak tertawa ...
3,\n\nSalut buat para saudara saudara ku yang be...,kaskus,0,0,1,0,dan nsalut buat para saudara saudara ku yang b...
4,"Yah lu min,, lu mau goblokin org,, tpi lu mala...",instagram,0,0,0,1,yah kamu administrator kamu mau goblokin orang...


In [24]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['processed_text']
        words = text.split()
        num_words = len(words)
        num_to_del = math.ceil(len(text.split(' ')) / 4)

        print("Index:", i)
        print("Original:", text, '\n')

        if num_to_del > 0:
            indices_to_delete = np.random.choice(num_words, size=num_to_del, replace=False)
            mask = np.ones(num_words, dtype=bool)
            mask[indices_to_delete] = False
            augmented_words = np.array(words)[mask]
            augmented = ' '.join(augmented_words)
        else:
            augmented = text  # if too short, don't change

        data.at[i, "processed_text"] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        # time.sleep(5)

Index: 1
Original: url menang berkat dua pemain muslim dan url enam pemain muslim yang membawa madrid juara dan url pria muslim di balik kemenangan memang madrid di final liga champions dan njink untung saudick kalah coba kalau menang pasti bawa bawa agama tertawa tolol dan nbelum lupa tahun dulu dan url pemain kafir heran arab saudi muslim dan tertawa 

Augmented: menang berkat dan url enam pemain muslim yang madrid juara dan url di balik kemenangan madrid di liga champions dan njink untung saudick kalah coba menang bawa bawa agama tertawa tolol dan nbelum lupa dan url pemain kafir arab saudi muslim dan
Index: 7
Original: merawat energi al maidah untuk kebangkitan umat demi kejayan negara kesatuan republik indonesia h maulid agung dan reuni akbar alumni reuniakbaralumnipic twiter com wvkfzejvwn 

Augmented: merawat al maidah kebangkitan umat demi negara republik indonesia h maulid agung dan reuni akbar alumni reuniakbaralumnipic com
Index: 9
Original: dan nkata tmn w juga yang bisa me

In [25]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 151


In [26]:
data.to_csv('./noisy/word-deletion.csv', index=False)

# WORD INSERTION

In [28]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan menyisipkan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- sebar 7-10 kata tambahan di seluruh kalimat
- jangan mengubah kata yang sudah ada, hanya sisipkan kata baru
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [29]:
data = load_data()
data.head()

,original_text,source,pornografi,sara,radikalisme,pencemaran_nama_baik,processed_text
0,Kalo urusan THR kompak yee. Gak ada kafir kafi...,twitter,0,1,0,0,kalau urusan thr kompak ya tidak ada kafir kaf...
1,[url=&quot;https://m.republika.co.id/amp_versi...,kaskus,0,1,0,0,url menang berkat dua pemain muslim dan url en...
2,Pantatnya ga sempurna banyak lemak Haha.. Kata...,instagram,1,0,0,1,pantatnya tidak sempurna banyak lemak tertawa ...
3,\n\nSalut buat para saudara saudara ku yang be...,kaskus,0,0,1,0,dan nsalut buat para saudara saudara ku yang b...
4,"Yah lu min,, lu mau goblokin org,, tpi lu mala...",instagram,0,0,0,1,yah kamu administrator kamu mau goblokin orang...


In [30]:
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['processed_text']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["processed_text"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(5)

Index: 1
Original: url menang berkat dua pemain muslim dan url enam pemain muslim yang membawa madrid juara dan url pria muslim di balik kemenangan memang madrid di final liga champions dan njink untung saudick kalah coba kalau menang pasti bawa bawa agama tertawa tolol dan nbelum lupa tahun dulu dan url pemain kafir heran arab saudi muslim dan tertawa 

Augmented: url sekali menang berkat sangat dua pemain muslim dan url enam pemain muslim yang membawa madrid juara dan url pria muslim di balik kemenangan memang madrid di final liga champions dan nanti njink untung saudick kalah coba kalau menang pasti bawa bawa agama kenapa tertawa tolol dan mungkin nbelum lupa tahun dulu dan url pemain cepat kafir heran lalu arab saudi muslim dan disana tertawa biru


In [ ]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 151


In [ ]:
data.to_csv('./noisy/word-insertion.csv', index=False)

# COMBINED

In [8]:
prompt_template = f'''KALIMAT:

- lakukan 3 noise injection: typo, penghapusan kata, atau penyusupan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- lakukan noise injection pada maksimum 40% kata dari kalimat
- pastikan kalimat akhir memiliki masing-masing macam noise injection
- jika ada beberapa kata yang sama, hanya ubah satu kata saja
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil
- output hanya berupa kalimat akhir yang telah diubah
'''

In [9]:
data = load_data()
data.head()

,original_text,source,pornografi,sara,radikalisme,pencemaran_nama_baik,processed_text
0,Kalo urusan THR kompak yee. Gak ada kafir kafi...,twitter,0,1,0,0,kalau urusan thr kompak ya tidak ada kafir kaf...
1,[url=&quot;https://m.republika.co.id/amp_versi...,kaskus,0,1,0,0,url menang berkat dua pemain muslim dan url en...
2,Pantatnya ga sempurna banyak lemak Haha.. Kata...,instagram,1,0,0,1,pantatnya tidak sempurna banyak lemak tertawa ...
3,\n\nSalut buat para saudara saudara ku yang be...,kaskus,0,0,1,0,dan nsalut buat para saudara saudara ku yang b...
4,"Yah lu min,, lu mau goblokin org,, tpi lu mala...",instagram,0,0,0,1,yah kamu administrator kamu mau goblokin orang...


In [8]:
random.seed(42)
augmented_cnt = 0

for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['processed_text']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["processed_text"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(3.5)

Index: 1
Original: url menang berkat dua pemain muslim dan url enam pemain muslim yang membawa madrid juara dan url pria muslim di balik kemenangan memang madrid di final liga champions dan njink untung saudick kalah coba kalau menang pasti bawa bawa agama tertawa tolol dan nbelum lupa tahun dulu dan url pemain kafir heran arab saudi muslim dan tertawa 

Augmented: url menang brkat dua muslim dan url enam pemain muslim yang membawa apel madrid juaraa dan url muslim di balik kemenangan biru memanng madrid di liga champions dan njink robot untungg kalah coba kalau menang pasti kucing bawaa bawa tertawa tolol es dan nbelum luppa dulu dan url pemain kafir batu hiran arab muslim dan tertawa
Index: 7
Original: merawat energi al maidah untuk kebangkitan umat demi kejayan negara kesatuan republik indonesia h maulid agung dan reuni akbar alumni reuniakbaralumnipic twiter com wvkfzejvwn 

Augmented: merawt energi al maidah kebangkitan umat pisang demi kejayen kesatuan biru republik indonsia h ma

ServerError: 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}

In [9]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 100


In [10]:
data.to_csv('./noisy/combined-noise.csv', index=False)

In [10]:
random.seed(42)
indexes = [1061, 1062, 1064, 1073, 1085, 1091, 1093, 1094, 1125, 1133, 1139, 1148, 1149, 1165, 1168, 1174, 1182, 1184, 1185, 1193, 1199, 1236, 1240, 1244, 1246, 1261, 1280, 1281, 1316, 1322, 1373, 1374, 1379, 1391, 1392, 1401, 1408, 1417, 1419, 1429, 1437, 1446, 1450, 1452, 1477, 1482, 1489, 1496, 1505, 1506, 1538, 1543]
augmented_cnt = 100

for i in indexes:
    # n = random.uniform(0, 1)

    # if n <= 0.1:
    augmented_cnt += 1
    text = data.iloc[i]['processed_text']

    print("Index:", i)
    print("Original:", text, '\n')

    augmented = generate_noise(build_prompt(text))
    data["processed_text"].values[i] = augmented

    print("Augmented:", augmented)
    print("=" * 50)

    time.sleep(3.5)

Index: 1061
Original: dan dan baca berita yang dicabut kasus apa tong dan dan nasbung gobloknya keterlaluan ini kasus cabul kok dibawa bawa motret nbaca yang nyinyir tong dan nsori iya tong tidak sudi jadi nasbung apalagi nastak dan nyg dicabut soal penghinan pancasila yang dilaporin sih sukmawati adik megawati jungjungan jokowi dan ngw tulis untuk nyentil orang yang berkomentar bilang mau lanjut polisin soal chat mesum dan dan nlo pikir aku bego apa kamu mau adu pinter sama aku dan nastak ngehek selalu brasa paling pinter dan 

Augmented: dan dan apel berit yang dicabut kasuz bola tong dan dan nasbung gobloknya ketrlaluan kasus cabul merah dibawwa bawa motred langit yang ninyir tong dan tong tidk sudi nasbung nastak biru dan nyg mobil dicabut soal penghinann pancasila hijau yang dilaporinn sih sukmawati megawati jungjungan jokowi kucing dan ngw tulis untuk nyentil orang yang berkomentar bilang mau lanjut polisin soal chatt anjing mesum dan dan nlo pikir aku bego apa kamu mau adu pinte

In [11]:
data.to_csv('./noisy/combined-noise-2.csv', index=False)